In [39]:
import json
import re
import pandas as pd

pd.set_option('display.max_colwidth', None)

with open(r"TVs-all-merged.json", "r") as file:
    data = json.load(file)
data


{'29PFL4508/F7': [{'shop': 'bestbuy.com',
   'url': 'http://www.bestbuy.com/site/Philips+-+4000+Series+-+29%26%2334%3B+Class+(28-1/2%26%2334%3B+Diag.)+-+LED+-+720p+-+60Hz+-+HDTV/9806538.p;template=_specificationsTab',
   'modelID': '29PFL4508/F7',
   'featuresMap': {'Screen Size Class': '29"',
    'DVI Inputs': '0',
    'ENERGY STAR Certified': 'Yes',
    'Ethernet Port': 'No',
    'Sleep Timer': 'Yes',
    'Width': '27 inches',
    'USB Port': 'Yes',
    'TV Type': 'LED Flat-Panel',
    'V-Chip': 'Yes',
    'Component Video Inputs': '1',
    'UPC': '609585230651',
    'Screen Size (Measured Diagonally)': '28-1/2"',
    'Mount Bracket/VESA Pattern': '100mm x 100mm',
    'PC Inputs': '1',
    'Aspect Ratio': '4:3 and 16:9',
    'Audio Outputs': '1',
    'Sound Leveler': 'Yes',
    'Media Card Slot': 'No',
    'Weight': '12.1 lbs. with stand (10.6 lbs. without)',
    'Vertical Resolution': '720p',
    'Composite Inputs': '1',
    'Brightness': '270 cd/mÂ²',
    'Watts/Channel': '8',
    

In [43]:
# clean two types of data: 1: title data, 2:key-value pairs
df = pd.DataFrame(columns=["model ID","shop", "Title"])
duplicate_models = []
for key, values in data.items():
    # new_row = pd.DataFrame({"model ID": [values[0]["modelID"]],"shop": values[0]["shop"], "Title": values[0]["title"].lower()})
    # df = pd.concat([df,new_row], ignore_index = True)  
    if len(values) > 1:
        duplicate_models.append(key)
len(duplicate_models)

329

### Investigate different features

In [ ]:
all_features = set()
for model in data.values():
    for product in model:
        features = product["featuresMap"]
        all_features.update(features.keys())

all_features = sorted(all_features)
df_all_features = pd.DataFrame(all_features, columns=["Fearture Keys", "values"])
df_all_features.to_excel("unique_feature_keys.xlsx", index=False)

3-D -> 3D
remove ":"
for hdmi: only get the number of text

In [206]:
def clean_feature_keys(string):
    # convert to all lowercase and some features end with ":"
    string = string.strip(":").lower()
    
    # Multiple HDMI input features:
    hdmi_variations = ["hdmi", "hdmi connection", "hdmi input", "hdmi inputs"]
    for value in hdmi_variations:
        string = re.sub(value, "hdmi", string)
    
    # Multiple color names
    color_variations = ["color", "cabinet color", "color name"]
    for value in color_variations:
        string = re.sub(value, "color", string)

    # Multiple brand Names 
    brand_variations = ["brand name", 'brand']
    for value in brand_variations:
        string = re.sub(value, "brand", string)

    return string

clean_feature_keys(" brand name cabinet color skdjfwel;f hdmi connection kwef")


' brand color skdjfwel;f hdmi kwef'

In [ ]:
def clean_feature_values_old(string):
    string = string.lower()
    # Normalization Brand Names

    # type error 
    string = re.sub("pansonic", "panasonic", string)

    # some brand names(Spectre) have " Inc."
    string = re.sub(" inc.", "", string)

    # lG has it in brand name, electronics sometimes occurs in "url" or "meta title", but we dont focus on those keys
    string = re.sub("lg electronics", "lg", string)

    # JVC TV replace by JVC
    string = re.sub("jvc tv", "jvc", string)

    # Normalize units

    # Inches -> inch (this one is sometimes in product dimensions)
    placeholder = "placeholder"
    # inch_variations = [" inches", "inches" , r'\"', "-inch", " inch", "inch"]
    inch_variations = [r'inches', r'["\”]', r'-inch', r' inch', r'inch', r'\binch\b']
    for value in inch_variations:
        string = re.sub(value, placeholder, string)
        print(string)

    string = re.sub(placeholder, "inch", string)
    # string = string.replace(placeholder, "inch")

    # hertz -> hz
    hz_variations = [" hertz", "hertz", "  hz", " hz", "-hz", "hz"]
    for value in hz_variations:
        string = re.sub(value, "hz", string)

    # pounds -> lbs, 
    # WHATCH OUT THAT LB ISNT IN A SENTENCE 
    lbs_variations = ["pounds", "pound", "lbs.", " lbs", "lbs", " lb", "lb"]
    placeholder = "placeholder"
    for value in lbs_variations:
        string = re.sub(value, placeholder, string)
    string = re.sub(placeholder, "lbs", string)

    # wireless key: wifi variations -> wifi
    wifi_variations = ["wifi ready", "wi-fi built-in", "wifi built-in", "built-in wifi","wi-fi"]
    for value in wifi_variations:
        string = re.sub(value, "wifi", string)
    
    return string

In [299]:

def clean_feature_values(string):
    string = string.lower()
    
    # Normalize Brand Names
    string = re.sub(r"pansonic", "panasonic", string)
    string = re.sub(r" inc\.", "", string)
    string = re.sub(r"lg electronics", "lg", string)
    string = re.sub(r"jvc tv", "jvc", string)

    # Normalize units
    # Inches -> inch
    inch_variations = [r'\binches\b', r'["\”]', r'-inch', r'\b inch\b', r'\binch\b']
    for value in inch_variations:
        string = re.sub(value, "inch", string)
    
    # Hertz -> hz
    hz_variations = [r'\b hertz\b', r'\bhertz\b', r'\b hz\b', r'\bhz\b', r'-hz']
    for value in hz_variations:
        string = re.sub(value, "hz", string)

    # Pounds -> lbs
    lbs_variations = [r'\bpounds\b', r'\bpound\b', r'\blbs\.\b', r'\b lbs\b', r'\blbs\b', r'\b lb\b', r'\blb\b']
    for value in lbs_variations:
        string = re.sub(value, "lbs", string)

    # Wifi variations -> wifi
    wifi_variations = [r'wifi ready', r'wi-fi built-in', r'wifi built-in', r'built-in wifi', r'wi-fi']
    for value in wifi_variations:
        string = re.sub(value, "wifi", string)
    
    return string


### clean data

In [298]:
def clean_data(input, output):
    # Open file
    with open(input, "r") as file:
        data = json.load(file)

    # copy file
    data_copy = data.copy()

    # create new dict, since we cant change existing and iterate over it
    for products in data_copy.values():
        for tv in products:
            new_tv = {}
            for key, value in tv.items():
                if key == "featuresMap":
                    new_features_map = {}
                    for feature_key, feature_value in tv[key].items():
                        cleaned_feature_key = clean_feature_keys(feature_key)
                        cleaned_feature_value = clean_feature_values(feature_value)
                        new_features_map[cleaned_feature_key] = cleaned_feature_value
                    new_tv[key] = new_features_map
                else:
                    cleaned_key = clean_feature_keys(key)
                    cleaned_value = clean_feature_values(value)
                    new_tv[cleaned_key] = cleaned_value

            tv.clear()
            tv.update(new_tv)

    with open(output, "w") as output_file:
        # indent =4 to get the same structure as original data
        json.dump(data_copy, output_file, indent=4)

clean_data("TVs-all-merged.json", "TVs-all-merged-cleaned.json")


#### Brand names
Idea add brand names as model words, extract them from the title or from feature map. We can do a combination of model words as 

In [293]:
string = "20.4 inches"

clean_feature_values(string)


'20.4es'

In [224]:
# get all brand names from "brand" key in featuresMap
def get_all_brand_names(data):
    possible_brands = set()
    for model in data.values():
        for product in model:
            for key, value in product["featuresMap"].items():
                if key == "brand":
                    possible_brands.add(value)
    # add popular tv names which do not come up in the brand keys
    possible_brands.update(["insignia", "tcl", "dynex"])
    return possible_brands

In [225]:
def get_tv_brand(title, data_var):
    possible_brands = list(get_all_brand_names(data_var))
    brands_found = []
    titles = []
    for brand in possible_brands:

        # make sure brand name is not used in different cotext
        if type(brand) != str:
            print(brand)
        pattern = rf'(?<![a-zA-Z]){re.escape(brand)}(?![a-zA-Z])'
        if re.search(pattern, title):
            brands_found.append(brand)
            titles.append(title)
    
    if len(brands_found) == 1:
        return brands_found[0]
    elif len(brands_found) >= 1:
        print("there are multiple brands found in 1 TITEL: ", brands_found, titles)
        
        # bc there is a high chance that a small letter brands are out of context, pick the longest one (only in 1 case "panasonic tcl1...")

        longest_brand = brands_found[0]
        for i in range(len(brands_found) - 1):
            if len(brands_found[i+1]) > len(longest_brand):
                longest_brand = brands_found[i]
        return longest_brand
    else:
        return 0

In [ ]:
# Example usage
# with open("TVs-all-merged-cleaned.json", "r") as file:
#     data_cleaned = json.load(file)

# title = "This TV has a bulge and is of the brand samsung."
# possible_brands = list(get_all_brand_names(data_cleaned))
# found_brands = []
# for brand in possible_brands:
#     pattern = rf'(?<![a-zA-Z]){re.escape(brand)}(?![a-zA-Z])'
#     if re.search(pattern, "this is a samsung nd lg tv"):
#         print(brand)
#         found_brands.append(brand)

# found_brands


lg
samsung


['lg', 'samsung']

In [221]:
get_tv_brand("lg 26inch class 26inch diag. ledlcd tv 720p hdtv glossy black, matte black 26lt560c - best buy",data_cleaned)

'lg'

In [300]:
def include_brand_in_features(input_file, output_file):
    with open(input_file, "r") as file:
        data_cleaned = json.load(file)

    # titles_not_found = pd.DataFrame(columns=["title"])
    for model in data_cleaned.values():
        i = 0
        for tv in model: 
            if "brand" not in tv["featuresMap"].keys():
                title = tv["title"]
                brand = get_tv_brand(title, data_cleaned)

                # if brand == 0:
                #     # print(f"brand not found in title: {title}")
                #     new_row = pd.DataFrame({"title": [title]})
                #     titles_not_found = pd.concat([titles_not_found, new_row], ignore_index=True)

                if brand != 0:
                    tv["featuresMap"]["brand"] = brand
    
    # titles_not_found.to_excel("not found.xlsx")
    with open(output_file, "w") as o_file:
        json.dump(data_cleaned, o_file, indent=4)



include_brand_in_features("TVs-all-merged-cleaned.json", "TVs-all-merged-cleaned-brands.json")           

there are multiple brands found in 1 TITEL:  ['panasonic', 'tcl'] ['panasonic tcl47et5 - best buy', 'panasonic tcl47et5 - best buy']


In [204]:
with open("TVs-all-merged-cleaned-brands.json", "r") as file:
    data_cleaned_brands = json.load(file)

titles = list()  # Initialize an empty list for titles

for model in data_cleaned_brands.values():
    for tv in model:
        if "brand" not in tv["featuresMap"].keys():
            titles.append(tv["title"])  # Append titles correctly

# Create a DataFrame and save to an Excel file
df = pd.DataFrame(titles, columns=["title"])  # Convert titles list to DataFrame
df.to_excel("no_brands.xlsx", index=False)  # Save to Excel